Note: some of the starting code was built upon notebooks developed for the FMA dataset, found at the FMA dataset github: https://github.com/mdeff/fma


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

from sklearn import metrics
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)

In [2]:
features = pd.read_csv("fma_metadata/features.csv", header=[0,1,2], index_col=0)
tracks = pd.read_csv("fma_metadata/tracks.csv", header = [0,1], index_col=0)

# echonest = pd.read_csv("fma_metadata/echonest.csv", header = [0,1,2], index_col=0)
# genres = pd.read_csv("fma_metadata/genres.csv", index_col=0)

In [3]:
features.shape, tracks.shape #, echonest.shape, genres.shape

((106574, 518), (106574, 52))

In [4]:
features.sample(10)

feature    chroma_cens                                                     \
statistics    kurtosis                                                      
number              01        02        03         04        05        06   
track_id                                                                    
30327        -0.056403  0.572917 -0.252751  -0.082944 -0.217285  0.606812   
64886         0.202467 -0.103764 -0.150173  -0.027986 -0.574337 -0.677594   
10363        -0.521812 -0.069861  0.233605  -0.358445 -0.194005 -0.235722   
69238        -0.271200 -0.912395 -0.603931   0.901456  2.804582 -1.364159   
99933        -0.740334  0.145557 -0.450127  -0.466761 -0.429336 -0.669380   
112113       -0.207235 -0.224607 -1.027846   0.271164 -0.413899  1.163566   
60761        -0.397428 -0.405496 -0.202924   1.074359  5.086401 -0.422976   
111087       -1.869139 -1.754297  9.770545  10.016171  5.433052  2.250412   
130131       -0.066390 -0.349301 -0.457758  -1.082688  1.247539 -0.812264   
136744        0.175255 -0.482550 -0.493647  -0.484613  0.796493 -0.417245   

feature                                                                  \
statistics                                                                
number            07        08        09        10         11        12   
track_id                                                                  
30327      -0.115844  0.665620  0.956418  0.233289   0.628541 -0.271816   
64886      -0.027608 -0.271431 -0.233017 -0.466271  -0.426682  0.196158   
10363       0.053286 -0.277203 -0.361728 -0.396846  -0.902609 -0.553818   
69238      -1.351982  0.185306 -0.303396 -0.993504  -0.112533 -0.010669   
99933       0.487048 -0.318724 -0.333138 -0.501481   0.595084 -1.187851   
112113     -0.258034 -0.025644  1.445553 -1.128730  -0.569559 -0.694576   
60761      -0.815818 -0.348202 -0.730211 -0.708499  -0.316378 -0.168084   
111087     -1.434503 -0.879450 -1.209927  2.101183  82.589218 -0.837203   
130131     -0.938598 -0.142981 -0.556337 -0.428071   0.046147  4.518015   
136744      0.376324 -0.064748  0.140697 -0.655926   0.357117 -0.259136   

feature                                                                 \
statistics       max                                                     
number            01        02        03        04        05        06   
track_id                                                                 
30327       0.573116  0.520453  0.575064  0.499276  0.604245  0.654665   
64886       0.705363  0.619303  0.692371  0.675960  0.660625  0.654495   
10363       0.690568  0.674969  0.678100  0.693173  0.632949  0.661484   
69238       0.500588  0.669439  0.483760  0.516371  0.506514  0.555908   
99933       0.691688  0.613607  0.694977  0.639602  0.703880  0.562888   
112113      0.707348  0.668689  0.636916  0.571590  0.620634  0.606658   
60761       0.589996  0.462798  0.452413  0.492357  0.588041  0.685986   
111087      0.816979  0.667440  0.447044  0.389923  0.437807  0.367328   
130131      0.572585  0.690583  0.710258  0.568014  0.699027  0.658681   
136744      0.559218  0.511762  0.481011  0.530589  0.617237  0.529808   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
30327       0.559836  0.628513  0.645296  0.619581  0.666999  0.518753   
64886       0.633511  0.677464  0.582604  0.617097  0.637982  0.640624   
10363       0.648511  0.685943  0.650322  0.666105  0.678571  0.674934   
69238       0.706679  0.644790  0.692941  0.577273  0.626802  0.677786   
99933       0.561036  0.475861  0.452571  0.430492  0.610863  0.656217   
112113      0.654729  0.693965  0.577630  0.701985  0.644803  0.661850   
60761       0.693587  0.514681  0.549691  0.569955  0.632906  0.699934   
111087      0.63846

In [5]:
tracks.sample(10)

album                                            \
         comments         date_created        date_released   
track_id                                                      
60975           0  2012-02-24 19:25:26  2011-03-16 00:00:00   
42272           0  2008-11-26 01:45:05  2009-01-06 00:00:00   
104691          0  2014-05-02 16:15:40  2014-05-01 00:00:00   
43870           1  2011-02-19 14:50:58  2011-02-19 00:00:00   
114414          3  2015-02-06 13:10:45  2012-03-12 00:00:00   
63126           0  2012-04-02 16:47:09  2012-04-02 00:00:00   
57647           0  2011-12-15 17:28:48  2011-12-12 00:00:00   
137715          0  2016-05-24 00:48:17                  NaN   
17946           0  2009-08-12 09:07:11  2009-08-10 00:00:00   
62338          -1                  NaN                  NaN   

                                                            \
                                 engineer favorites     id   
track_id                                                     
60975                                 NaN         2  11156   
42272                                 NaN         2      4   
104691    Ernie Indradat with Erin Tonkon         0  16411   
43870                                 NaN         0   8609   
114414                       Scott Holmes        11  17631   
63126                                 NaN         0  11444   
57647                        Irene Trudel         0  10610   
137715                                NaN         1  20690   
17946                Richard Rusincovitch         0   4109   
62338                                 NaN        -1     -1   

                                                                     \
                                                information listens   
track_id                                                              
60975     <p>LES BOIS ROUGES was originally released as ...    1414   
42272             <p> "spiritual songs" from Nicky Cook</p>    2710   
104691    <p>Kristeen Young returnes to WFMU to chat wit...    3226   
43870     <p><span style="font-family:Times;font-size:me...     739   
114414    <p> </p>\n<p> </p>\n<p><span style="font-size:...   71946   
63126     <p>From August 2010 to January 2012, New Weird...    6845   
57647     <p><span style="font-family:Verdana, Geneva, A...    6374   
137715                                                  NaN   33846   
17946     <p><span style="font-family:Verdana, Geneva, A...    3005   
62338                                                   NaN      -1   

                                                                           \
              producer                                               tags   
track_id                                                                    
60975              NaN                                                 []   
42272              NaN                                                 []   
104691           Julie  ['electroacoustic', 'post-punk', 'kristeen you...   
43870              NaN                                                 []   
114414    Scott Holmes  ['acoustic', 'shoegaze', 'folk', 'filmscore', ...   
63126              NaN                                                 []   
57647     Irene Trudel                                                 []   
137715             NaN                                                 []   
17946         Liz Berg                                    ['new orleans']   
62338              NaN                                                 []   

                                                                    \
                                                      title tracks   
track_id                                                             
60975                                       Les Bois Rouges      5   
42272                                                 Niris     13   
104691    Live on WFMU's Dark Night of the Soul with Jul...      9   
43870                                  

# EDA / Data Cleaning

### Questions:
- What are the top genres in this dataset? 
- What does the distribution of top genres look like? 
- How many "top genres" are NA?
- What to do with the huge class imbalance (counts differenct a factor of 100 - 1000)?

### Observations:
- The "tracks" dataset is organized with a top category (ie album, artist, set, track) and subcategories, so columns must be accessed in this way (see next cell as an example)
- Over HALF of the tracks do not have value in "genre_top" column

#### the full dataset has NaN for about half of the genre_top features

In [6]:
# Top Genres list for the entire set 
tracks.loc[:,('track','genre_top')].value_counts(dropna=False)

NaN                    56976
Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

In [7]:
# Create a large subset that drops rows with NaN in the genre_top column
large = tracks[tracks['track','genre_top'].notna()]
large.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

#### Notice there's a huge class imbalance in the largest dataset. I trim it down below:

In [8]:
# create a large subset with only genres that have at least 1000 tracks 
over_1000 = ["Rock","Experimental","Electronic","Hip_Hop","Folk","Pop","Instrumental","International","Classical"]
large_b = large[large['track','genre_top'].isin(over_1000)]
large_b.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock             14182
Experimental     10608
Electronic        9372
Folk              2803
Pop               2332
Instrumental      2079
International     1389
Classical         1230
Name: (track, genre_top), dtype: int64

#### Grouping features to be used with the classification models

In [9]:
# What are the major features in the "features" dataset?
feature_list = features.columns.levels[0].tolist()
feature_list

['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr']

In [10]:
# Add a few feature COMBINATIONS (based on EDA done in the MVP)

# Top 2 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast'])
# Top 4 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast','chroma_cqt','chroma_stft'])
# All the features, EXCLUDING the 4 features with the worst accuracy scores by SVC classification
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','spectral_centroid',
                     'spectral_contrast','tonnetz'])
# All the features!
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr'])

# Indexes for the tables
indexes = ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
           'spectral_contrast','spectral_rolloff','tonnetz','zcr',
           'mfcc/spectral_contrast',
           'mfcc/spectral_contrast/chroma_cqt/chroma_stft',
           'chroma_cens/chroma_cqt/chroma_stft/mfcc/spectral_centroid/spectral_contrast/tonnetz',
           'ALL']

In [11]:
feature_list


['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr',
 ['mfcc', 'spectral_contrast'],
 ['mfcc', 'spectral_contrast', 'chroma_cqt', 'chroma_stft'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'spectral_centroid',
  'spectral_contrast',
  'tonnetz'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'rmse',
  'spectral_bandwidth',
  'spectral_centroid',
  'spectral_contrast',
  'spectral_rolloff',
  'tonnetz',
  'zcr']]

# Data Analysis


In [12]:
# for lack of a better method, create a mask, LARGE, that only has "top_genres" with at least 1000tracks
large_b = ((tracks['track','genre_top'].notna()) & (tracks['track','genre_top'].isin(over_1000)))
large_b

track_id
2         False
3         False
5         False
10         True
20        False
          ...  
155316     True
155317     True
155318     True
155319     True
155320    False
Name: (track, genre_top), Length: 106574, dtype: bool

In [15]:
accuracy_scores = pd.DataFrame(index=indexes)

# add first column that shows the number of dimensions for each feature (or collection of features)
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
accuracy_scores['dim']=dim_col

accuracy_scores

,dim
chroma_cens,84
chroma_cqt,84
chroma_stft,84
mfcc,140
rmse,7
spectral_bandwidth,7
spectral_centroid,7
spectral_contrast,49
spectral_rolloff,7
tonnetz,42


In [18]:
recall_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
recall_scores['dim']=dim_col


In [19]:
f1_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
f1_scores['dim']=dim_col


### Logistic Regression

In [20]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['lr'] = accuracy_col
recall_scores['lr'] = recall_col
# f1_scores['lr'] = f1_col

ValueError: Cannot index with multidimensional key

#### Weighted Logistic Regression

In [21]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr_weighted = LogisticRegression(max_iter=1000,class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
    lr_weighted.fit(X_train, y_train)
    y_pred = lr_weighted.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted lr'] = accuracy_col
# recall_scores['weighted lr'] = recall_col
# f1_scores['weighted lr'] = f1_col

ValueError: Cannot index with multidimensional key

### K Nearest Neighbors

In [22]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, (feature)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=200)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['knn'] = accuracy_col
# recall_scores['knn'] = recall_col
# f1_scores['knn'] = f1_col

ValueError: Cannot index with multidimensional key

### Decision Tree

In [23]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    DT = DecisionTreeClassifier(max_depth=5)
    DT.fit(X_train, y_train)
    y_pred = DT.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['dt'] = accuracy_col
# recall_scores['dt'] = recall_col
# f1_scores['dt'] = f1_col

ValueError: Cannot index with multidimensional key

### Random Forest

In [24]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    RF = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['rf'] = accuracy_col
# recall_scores['rf'] = recall_col
# f1_scores['rf'] = f1_col

ValueError: Cannot index with multidimensional key

### Ada Boost

In [25]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    ada = AdaBoostClassifier(n_estimators=10)
    ada.fit(X_train, y_train)
    y_pred = ada.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['ada'] = accuracy_col
# recall_scores['ada'] = recall_col
# f1_scores['ada'] = f1_col

ValueError: Cannot index with multidimensional key

### Naive Bayes

In [26]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    nbc_base = GaussianNB()
    nbc_base.fit(X_train, y_train)
    y_pred = nbc_base.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['nb'] = accuracy_col
# recall_scores['nb'] = recall_col
# f1_scores['nb'] = f1_col

ValueError: Cannot index with multidimensional key

### SVC Classifier

In [27]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

ValueError: Cannot index with multidimensional key

In [28]:
accuracy_scores

,dim
chroma_cens,84
chroma_cqt,84
chroma_stft,84
mfcc,140
rmse,7
spectral_bandwidth,7
spectral_centroid,7
spectral_contrast,49
spectral_rolloff,7
tonnetz,42


In [29]:
accuracy_scores.to_excel("accuracyScores.xlsx")

#### Weighted SVC classifier

In [30]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf',class_weight={'International' : 4, 'Classical' : 4, 'Instrumental':3,'Folk':2, 'Pop':2}solver='liblinear')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted svc'] = accuracy_col
# recall_scores['weighted svc'] = recall_col
# f1_scores['weighted svc'] = f1_col

SyntaxError: invalid syntax (<unknown>, line 12)

#### SVC optimized with grid search

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(C=10,kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

In [ ]:
accuracy_scores

#### Optimize SVC with grid search

In [ ]:
# I couldn't fully run this. Took too many hours.

y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, 'mfcc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)


# defining parameter range 
param_grid = {'C': [0.01,0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 

# fitting the model for grid search 
grid.fit(X_train, y_train)

# print best parameter after tuning 
print(grid.best_params_) 
y_pred = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_scores.to_excel("accuracy_scores.xlsx")

# Visualizations

## Confusion Matrix

In [ ]:
# Logistic Regression with all of the features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
sorted(tracks.loc[large_b,('track','genre_top')].unique())

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Logistic Regression with weighted genres and all features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr_weighted = LogisticRegression(class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
lr_weighted.fit(X_train, y_train)
y_pred = lr_weighted.predict(X_test)
print("weighted lr accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("weighted lr recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("weighted lr f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Confusion matrix with SVM classifier, all genres
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)
SVCrbf = SVC(kernel='rbf')
SVCrbf.fit(X_train, y_train)
y_pred = SVCrbf.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0)
plt.savefig('svc confusion_matrix'+'.jpg');

## PCA plots

In [ ]:
# Two genres that are RARELY misclassified as each other
# Note: this code was loosely copied from the FMA "usage" notebook (https://nbviewer.org/github/mdeff/fma/blob/outputs/usage.ipynb)
# all starting materials found at their github: https://github.com/mdeff/fma

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'spectral_contrast']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical spectral contrast', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Folk'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Folk vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical all features', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Classical'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Electronic vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are SOMETIMES misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Electronic'
genre2 = tracks['track', 'genre_top'] == 'Folk'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Folk', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Rock', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Experimental'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Experimental'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Electronic', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape
